In [1]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

import numpy as np
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
C:\Users\PC\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Cuarto paso: correr el modelo

In [2]:
l_rates = [0.001, 0.01]
epochs = [10, 20, 30] 
batch_sizes = [16, 32]

experiment = 7 #Número donde inician los experimentos
for batch in batch_sizes:
    for epoch in epochs:
        for learning_rate in l_rates:
        
            print("*"*20)
            print("EXPERIMENTO: Batch:", batch, "Epoc: ", epoch, "learning_rate: ", learning_rate)
            
            #Cargar los parametros cada que inicia un experimento
            params = utils.yaml_to_dict(os.path.join('..','config.yml'))
            params['data_dir'] = os.path.join('..',params['data_dir'])
            
            experiment_name = 'Experimento1-Configuracion'+str(experiment)
            params['model_dir'] = os.path.join('..','experiments-weights',experiment_name)
            params['batch_size'] = batch
            params['num_epochs'] = epoch
            params['learning_rate'] = learning_rate
            
            #Limpiar Sesión
            tf.keras.backend.clear_session()
            
            #Cargar el modelo
            inputs = tf.keras.layers.Input(shape=(110,480,3))
            net = model.ModelArchitecture(num_classes=params['num_classes'])
            x = net(inputs, training=False)
            
            #Generar conjuntos de entrenamiento y validación y llevarlos a un generador
            preprocessing.split_data(params)
            train_generator = data.make_datagenerator(params, mode='training')
            val_generator = data.make_datagenerator(params,mode='validation')
            
            #Crear el directorio que guarda los checkpoints
            if not os.path.exists(params['model_dir']):
                os.makedirs(params['model_dir'])

            #Definir los callbacks
            cp_callback = tf.keras.callbacks.ModelCheckpoint(
                os.path.join(params['model_dir'], 'tf_ckpt'), 
                save_weights_only=True, 
                verbose=1,
                period=5)

            tb_callback = tf.keras.callbacks.TensorBoard(
                os.path.join(params['model_dir'], 'logs'))

            #Definir la función de optimización
            optimizer = tf.keras.optimizers.Adam(params['learning_rate'])

            #Definir los pasos para entrenamiento y validación
            steps_per_epoch = train_generator.n // params['batch_size']
            validation_steps = val_generator.n // params['batch_size']

            #Entrenar el modelo
            net.compile(optimizer=optimizer, loss=params['loss'], metrics=['sparse_categorical_accuracy'])
            net.fit_generator(
                train_generator, 
                steps_per_epoch=steps_per_epoch, 
                epochs=params['num_epochs'],
                workers=4,
                validation_data=val_generator, 
                validation_steps=validation_steps,
                callbacks=[cp_callback,tb_callback])
            
    
            #Obtener las métricas de resultados en el conjunto de validación
            params['batch_size'] = val_generator.n
            params['shuffle'] = False
            val_generator = data.make_datagenerator(params,mode='validation')
            Y_pred = net.predict_generator(val_generator, steps=1)
            y_pred = np.argmax(Y_pred, axis=1)
            
            print(confusion_matrix(val_generator.data, y_pred))
            target_names = ["arranca", "acelera", "pita", "izquierda","frena", "detente", "retrocede", "gira", "avanza", "derecha"]
            print(classification_report(val_generator.data, y_pred, target_names=target_names))
            
            #Aumentar el contador de experimentos
            experiment += 1

********************
EXPERIMENTO: Batch: 16 Epoc:  10 learning_rate:  0.001
Found 2960 images.
Found 30 images.
Epoch 1/10
185/185 [==============================] - 14s 77ms/step - loss: 2.3011 - sparse_categorical_accuracy: 0.1064 - val_loss: 2.3026 - val_sparse_categorical_accuracy: 0.1250
Epoch 2/10
185/185 [==============================] - 13s 68ms/step - loss: 2.2990 - sparse_categorical_accuracy: 0.1166 - val_loss: 2.3080 - val_sparse_categorical_accuracy: 0.1250acc
Epoch 3/10
185/185 [==============================] - 13s 71ms/step - loss: 2.2978 - sparse_categorical_accuracy: 0.1166 - val_loss: 2.3012 - val_sparse_categorical_accuracy: 0.1250
Epoch 4/10
185/185 [==============================] - 12s 66ms/step - loss: 2.2951 - sparse_categorical_accuracy: 0.1166 - val_loss: 2.3004 - val_sparse_categorical_accuracy: 0.1250
Epoch 5/10
184/185 [============================>.] - ETA: 0s - loss: 2.2883 - sparse_categorical_accuracy: 0.1379
Epoch 00005: saving model to ..\experiment

C:\Users\PC\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Found 2960 images.
Found 30 images.
Epoch 1/10
185/185 [==============================] - 15s 82ms/step - loss: 2.3021 - sparse_categorical_accuracy: 0.1122 - val_loss: 2.2989 - val_sparse_categorical_accuracy: 0.0625s - loss: 2.3048 -  - ETA: 6s - loss: 2.3051 - sparse_categori
Epoch 2/10
185/185 [==============================] - 13s 70ms/step - loss: 2.2998 - sparse_categorical_accuracy: 0.1155 - val_loss: 2.2860 - val_sparse_categorical_accuracy: 0.0625curacy: 0.1
Epoch 3/10
185/185 [==============================] - 13s 69ms/step - loss: 2.3004 - sparse_categorical_accuracy: 0.1084 - val_loss: 2.2824 - val_sparse_categorical_accuracy: 0.1250
Epoch 4/10
185/185 [==============================] - 13s 68ms/step - loss: 2.3002 - sparse_categorical_accuracy: 0.1159 - val_loss: 2.2865 - val_sparse_categorical_accuracy: 0.1250
Epoch 5/10
184/185 [============================>.] - ETA: 0s - loss: 2.3005 - sparse_categorical_accuracy: 0.1053
Epoch 00005: saving model to ..\experiments-weig

185/185 [==============================] - 13s 71ms/step - loss: 2.1387 - sparse_categorical_accuracy: 0.1970 - val_loss: 2.0321 - val_sparse_categorical_accuracy: 0.3125
Epoch 9/20
185/185 [==============================] - 12s 63ms/step - loss: 2.1158 - sparse_categorical_accuracy: 0.2081 - val_loss: 1.9897 - val_sparse_categorical_accuracy: 0.1875
Epoch 10/20
184/185 [============================>.] - ETA: 0s - loss: 2.1053 - sparse_categorical_accuracy: 0.2130
Epoch 00010: saving model to ..\experiments-weights\Experimento1-Configuracion9\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
185/185 [==============================] - 13s 71ms/step - loss: 2.1046 - sparse_categorical_accuracy: 0.2132 - val_loss: 2.0192 - val_sparse_categorical_accuracy: 0.3125
Epoch 11/20
185/185 [==============================] - 8s 45ms/step - loss: 2.1005 - sparse_categorical_accuracy: 0.2179 - val_loss: 2.0165 - val_sparse_categorical_accuracy: 0.1875
Epoch 12/20
185/185 [=============

185/185 [==============================] - 14s 76ms/step - loss: 1.8450 - sparse_categorical_accuracy: 0.3280 - val_loss: 1.7986 - val_sparse_categorical_accuracy: 0.3125
Epoch 6/20
185/185 [==============================] - 12s 64ms/step - loss: 1.8249 - sparse_categorical_accuracy: 0.3466 - val_loss: 1.7876 - val_sparse_categorical_accuracy: 0.3125
Epoch 7/20
185/185 [==============================] - 13s 70ms/step - loss: 1.8150 - sparse_categorical_accuracy: 0.3429 - val_loss: 1.7326 - val_sparse_categorical_accuracy: 0.3750
Epoch 8/20
185/185 [==============================] - 11s 59ms/step - loss: 1.7961 - sparse_categorical_accuracy: 0.3524 - val_loss: 1.7498 - val_sparse_categorical_accuracy: 0.3125
Epoch 9/20
185/185 [==============================] - 14s 77ms/step - loss: 1.7945 - sparse_categorical_accuracy: 0.3581 - val_loss: 1.7440 - val_sparse_categorical_accuracy: 0.3750
Epoch 10/20
183/185 [============================>.] - ETA: 0s - loss: 1.7924 - sparse_categorical_ac

185/185 [==============================] - 15s 79ms/step - loss: 2.1786 - sparse_categorical_accuracy: 0.2179 - val_loss: 2.1884 - val_sparse_categorical_accuracy: 0.1875
Epoch 7/30
185/185 [==============================] - 13s 70ms/step - loss: 2.0604 - sparse_categorical_accuracy: 0.2682 - val_loss: 2.2487 - val_sparse_categorical_accuracy: 0.2500
Epoch 8/30
185/185 [==============================] - 13s 72ms/step - loss: 1.9744 - sparse_categorical_accuracy: 0.3122 - val_loss: 2.3031 - val_sparse_categorical_accuracy: 0.1875
Epoch 9/30
185/185 [==============================] - 13s 69ms/step - loss: 1.9082 - sparse_categorical_accuracy: 0.3220 - val_loss: 2.3456 - val_sparse_categorical_accuracy: 0.1875orical_accurac - ETA: 2s - loss: 1.9036 - sparse_categorical_accuracy: 0.323 - ETA: 2s - loss: 1.9042 - sparse_ca
Epoch 10/30
183/185 [============================>.] - ETA: 0s - loss: 1.8601 - sparse_categorical_accuracy: 0.3480
Epoch 00010: saving model to ..\experiments-weights\Ex

185/185 [==============================] - 14s 78ms/step - loss: 1.7251 - sparse_categorical_accuracy: 0.3936 - val_loss: 2.5624 - val_sparse_categorical_accuracy: 0.2500
Epoch 20/30
183/185 [============================>.] - ETA: 0s - loss: 1.7200 - sparse_categorical_accuracy: 0.4027
Epoch 00020: saving model to ..\experiments-weights\Experimento1-Configuracion11\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
185/185 [==============================] - 15s 83ms/step - loss: 1.7197 - sparse_categorical_accuracy: 0.4027 - val_loss: 2.5806 - val_sparse_categorical_accuracy: 0.2500
Epoch 21/30
185/185 [==============================] - 13s 68ms/step - loss: 1.7109 - sparse_categorical_accuracy: 0.4027 - val_loss: 2.5578 - val_sparse_categorical_accuracy: 0.3125
Epoch 22/30
185/185 [==============================] - 13s 70ms/step - loss: 1.6961 - sparse_categorical_accuracy: 0.4095 - val_loss: 2.6640 - val_sparse_categorical_accuracy: 0.3750
Epoch 23/30
185/185 [==========

Found 2960 images.
Found 30 images.
Epoch 1/30
185/185 [==============================] - 14s 77ms/step - loss: 2.3031 - sparse_categorical_accuracy: 0.1074 - val_loss: 2.2761 - val_sparse_categorical_accuracy: 0.1875
Epoch 2/30
185/185 [==============================] - 14s 73ms/step - loss: 2.3003 - sparse_categorical_accuracy: 0.1105 - val_loss: 2.2726 - val_sparse_categorical_accuracy: 0.1875
Epoch 3/30
185/185 [==============================] - 12s 67ms/step - loss: 2.2995 - sparse_categorical_accuracy: 0.1118 - val_loss: 2.2659 - val_sparse_categorical_accuracy: 0.1250
Epoch 4/30
185/185 [==============================] - 13s 70ms/step - loss: 2.3003 - sparse_categorical_accuracy: 0.1101 - val_loss: 2.2742 - val_sparse_categorical_accuracy: 0.1250
Epoch 5/30
183/185 [============================>.] - ETA: 0s - loss: 2.3004 - sparse_categorical_accuracy: 0.1124
Epoch 00005: saving model to ..\experiments-weights\Experimento1-Configuracion12\tf_ckpt

Consider using a TensorFlow opt

185/185 [==============================] - 12s 63ms/step - loss: 2.3001 - sparse_categorical_accuracy: 0.1105 - val_loss: 2.2661 - val_sparse_categorical_accuracy: 0.1250
Epoch 24/30
185/185 [==============================] - 11s 61ms/step - loss: 2.2998 - sparse_categorical_accuracy: 0.1037 - val_loss: 2.2597 - val_sparse_categorical_accuracy: 0.1250
Epoch 25/30
184/185 [============================>.] - ETA: 0s - loss: 2.2995 - sparse_categorical_accuracy: 0.1067
Epoch 00025: saving model to ..\experiments-weights\Experimento1-Configuracion12\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
185/185 [==============================] - 13s 70ms/step - loss: 2.2997 - sparse_categorical_accuracy: 0.1064 - val_loss: 2.2589 - val_sparse_categorical_accuracy: 0.1250
Epoch 26/30
185/185 [==============================] - 12s 64ms/step - loss: 2.3002 - sparse_categorical_accuracy: 0.1149 - val_loss: 2.2689 - val_sparse_categorical_accuracy: 0.1250
Epoch 27/30
185/185 [==========

UnboundLocalError: local variable 'outs' referenced before assignment